# 제너레이터(generator)

## 메모리 효율이 높은 이터러블 객체
- 리스트나 튜플처럼 for 문에서 사용할 수 있는 <u>이터러블 객체</u>
- 리스트나 튜플은 모든 요소를 메모리에 유지하기 때문에 요소가 많아질수록 메모리 사용량이 커지는 단점이 있음
- 제너레이터는 다음 요소가 필요한 순간에 새로운 요소를 생성해서 리턴한다.
- 즉, <u>요소 수와 관계없이 메모리 사용량을 적게 유지함</u>

## 예시
값을 계속해서 무한히 반환하는 제너레이터 `inf`를 만들어보자.
- 특징은 `yield` 식이 있고 이게 제너레이터의 표시다.
- 무한히 반환하는 이 작업은 모든 요소를 메모리에 저장하는 리스트나 튜플로는 불가능하다.<br>무한개의 요소를 가지고 있는 리스트나 튜플을 만들 수 없기에

In [1]:
def inf(n):
    while True:
        yield n

In [2]:
# for i in inf(3):
#     print(i)

# 구현

## 1. 제너레이터 함수
함수처럼 구현하면되는데 return 대신 yield를 쓴다.

In [3]:
def gen_function(n):
    print('start')
    while n:
        print(f'yield: {n}')
        yield n
        n -= 1

In [9]:
gen = gen_function(3)
gen

<generator object gen_function at 0x103e61eb0>

제너레이터 함수의 리턴값은 **제너레이터 이터레이터(generator-iterator)** 라 불리는 **이터레이터(iterator)** 이다.

In [10]:
next(gen)

start
yield: 3


3

위의 함수에 yield n까지만 실행되고 대기, 다음에 호출하면 n-=1을 실행하고 다시 yield까지 실행

In [11]:
next(gen)

yield: 2


2

In [12]:
next(gen)

yield: 1


1

In [13]:
next(gen)

StopIteration: 

- 이터레이터는 스페셜 메소드 `__next()__` 호출 시마다 함수 안의 처리가 다음 yield식까지 수행된다.
- 호출자(여기선 next())에게 yield식에 전달한 값을 반환하고, 그 시점에서의 상태를 유지한 채 다음 행에서 처리를 중지함
- 계속 `__next__()`가 호출되면 다음 행부터 처리를 시작하고 함수를 벗어나면 StopIteration 예외를 발생시킴

제너레이터 함수는 이터레이터를 반환하므로 for문, 컴프리헨션, 인수에 이터러블을 받는 함수 등에 사용할 수 있다.

In [14]:
# for문에 사용
for i in gen_function(3):
    print(i)

start
yield: 3
3
yield: 2
2
yield: 1
1


In [16]:
# 컴프리헨션에 사용
[i for i in gen_function(3)]

start
yield: 3
yield: 2
yield: 1


[3, 2, 1]

In [17]:
# 이터러블을 받는 함수에 사용
max(gen_function(5))

start
yield: 5
yield: 4
yield: 3
yield: 2
yield: 1


5

## 2. 제너레이터 식

리스트나 튜플 등의 이터러블을 컴프리헨션을 이용해 제너레이터를 만들 수 있음<br>
이를 제너레이터 식이라 부르며 정의 방법은 리스트컴프리헨션과 같으나 ()를 씀

In [20]:
x = [1,2,3,4,5]

# 리스트 컴프리헨션
# 모든 요소가 메모리에 저장됨
x_list = [i for i in x]
print(x_list)

# 제너레이터 객체를 반환함
x_gen = (i for i in x)
print(x_gen)

[1, 2, 3, 4, 5]
<generator object <genexpr> at 0x1049b29e0>


리스트를 만들면 제너레이터의 마지막 요소까지 적용해 리스트를 생성

In [21]:
list(x_gen)

[1, 2, 3, 4, 5]

함수 호출 시 전달한 아규먼트가 제너레이터 식 하나면 컴프리헨션의 ()를 생략할 수 있음

In [23]:
max(i+1 for i in x) # max((i+1 for i in x))와 같음

6

## 3. yield from 식

제너레이터의 내부에서 추가로 제너레이터를 만들 때 사용

예시) 이터러블 안의 이터러블을 변환하는 제너레이터를 만들어보자

In [24]:
def chain(iterables):
    for iterable in iterables:
        for v in iterable:
            yield v
            
iterables = ('coding', 'fun')
list(chain(iterables))

['c', 'o', 'd', 'i', 'n', 'g', 'f', 'u', 'n']

위에 함수를 yield from 식을 조합해 제너레이터 식으로 치환할 수 있음

In [25]:
def chain(iterables):
    for iterable in iterables:
        yield from (v for v in iterable)
        
list(chain(iterables))

['c', 'o', 'd', 'i', 'n', 'g', 'f', 'u', 'n']